In [35]:
from buffalotoprogrammachine import interpret
from programmachine import Register, Instructions

In [36]:
# The Program Machine program we are aiming for
src = """buffalo 2
Buffalo 0

JZ buffalo 4
DEC buffalo
INC Buffalo
JZ buffalo 0
"""

In [37]:
program = "Buffalo buffalo buffalo buffalo Buffalo buffalo buffalo. 🐃Swamp buffalo enjoy wallowing in mudholes they make with their horns🐃 Buffalo buffalo buffalo buffalo. Buffalo buffalo Buffalo buffalo buffalo buffalo buffalo buffalo buffalo buffalo Buffalo buffalo. Buffalo buffalo buffalo buffalo buffalo buffalo buffalo buffalo. Buffalo buffalo buffalo buffalo Buffalo buffalo Buffalo buffalo buffalo buffalo 🐃It's true!🐃. Buffalo buffalo Buffalo buffalo buffalo buffalo buffalo buffalo."

In [38]:
src = interpret(program)
print(src)

buffalo 2
Buffalo 0

JZ buffalo 4
DEC buffalo
INC Buffalo
JZ buffalo 0


In [39]:
instructions = Instructions(src)
print(f'Initial Register Values\n-----------------------')
registers = instructions.registers
for key, val in zip(registers.keys(),
                    [str(register.value) for register in registers.values()]):
                print(f'{key}: {val}')
print()
instructions_with_linenumbers = [f'{i+1}: {" ".join(instructions.instruction_list[i])}' for i in range(len((instructions.instruction_list)))]
instruction_print = '\n'.join(instructions_with_linenumbers)
print(f"""Instructions
------------
{instruction_print}
""")
print("Run program\n-----------")
instructions.interpret()

Initial Register Values
-----------------------
buffalo: 2
Buffalo: 0

Instructions
------------
1: JZ buffalo 4
2: DEC buffalo
3: INC Buffalo
4: JZ buffalo 0

Run program
-----------
line 1: register buffalo is not empty, doing nothing
line 2: decreasing register buffalo from 2 to 1
line 3: increasing register Buffalo from 0 to 1
line 4: register buffalo is not empty, doing nothing
line number out of range, halting

Final Register Values on Halt
-----------------------------
buffalo: 1
Buffalo: 1
